In [7]:
from orkg import ORKG, Hosts

In [8]:
orkg = ORKG(host=Hosts.PRODUCTION)

In [16]:
orkg.predicates.by_id(id='P7057')

(Success) {'id': 'P7057', 'label': 'has subProblem', 'description': None, 'created_at': '2020-01-29T17:10:44.528+01:00', 'created_by': '00000000-0000-0000-0000-000000000000', 'modifiable': True, '_class': 'predicate'}

In [15]:
orkg.predicates.get(q='subProblem',exact=False,size=30,sort='label',desc=True)

(Success) [{'id': 'P7057', 'label': 'has subProblem', 'description': None, 'created_at': '2020-01-29T17:10:44.528+01:00', 'created_by': '00000000-0000-0000-0000-000000000000', 'modifiable': True, '_class': 'predicate'}]

In [20]:
orkg.classes.by_id(id='Resource')

(Fail) b'{"status":404,"error":"Not Found","path":"/api/classes/Resource/","timestamp":"2025-06-27T12:27:56.54404209+02:00","message":"Class \\"Resource\\" not found."}'

In [21]:
orkg.classes.get_all(q='Resource', exact=False)

(Success) [{'id': 'C15002', 'label': 'Resource', 'uri': 'http://www.w3.org/ns/dcat#Resource', 'description': None, 'created_at': '2021-04-22T13:30:56.155639+02:00', 'created_by': '7adcfc45-b315-4df7-b9a5-010e8f1ca9cb', 'modifiable': True, '_class': 'class'}, {'id': 'Resources', 'label': 'Resource', 'uri': None, 'description': None, 'created_at': '2021-11-02T10:18:04.524891+01:00', 'created_by': '00000000-0000-0000-0000-000000000000', 'modifiable': True, '_class': 'class'}, {'id': 'C21057', 'label': 'XrayResource', 'uri': None, 'description': None, 'created_at': '2021-11-16T19:58:09.620608+01:00', 'created_by': '1bfbb658-6a8e-4a3a-86e4-2e24b021ba9b', 'modifiable': True, '_class': 'class'}, {'id': 'C58034', 'label': 'Resource Size', 'uri': 'https://w3id.org/semsys/ns/swemls#ResourceSize', 'description': None, 'created_at': '2023-03-15T14:11:20.502354Z', 'created_by': '61a04b37-e5a3-40a1-a767-81adf2ce706e', 'modifiable': True, '_class': 'class'}, {'id': 'C58035', 'label': 'Resource Type',

In [22]:
from rdflib import Graph

# Load the RDF schema
g = Graph()
g.parse("schema.rdf", format="xml")  # Or 'turtle'/'n3' depending on your file


<Graph identifier=Nb5646c1c5d6941b0a1c78fbd5a6e0eec (<class 'rdflib.graph.Graph'>)>

In [ ]:
from rdflib import RDF, RDFS, OWL
from sentence_transformers import SentenceTransformer, util

# Load schema
g = Graph()
g.parse("schema.rdf", format="xml")  # or 'ttl', depending on format

# Load transformer
model = SentenceTransformer('all-MiniLM-L6-v2')

# Step 1: Extract terms with labels/comments
seen = set()
terms = []

for s in set(g.subjects()):  # ensures unique subjects
    if s in seen:
        continue
    seen.add(s)

    label = g.value(s, RDFS.label)
    comment = g.value(s, RDFS.comment)

    type_ = g.value(s, RDF.type)
    if type_ == OWL.Class or type_ == RDFS.Class:
        kind = "Class"
    elif type_ in {RDF.Property, OWL.ObjectProperty, OWL.DatatypeProperty}:
        kind = "Property"
    else:
        kind = "Term"

    if label or comment:
        text = f"{kind}: {label or ''}. Comment: {comment or ''}"
        terms.append((s, text.strip()))

# Step 2: Encode terms
term_texts = [text for _, text in terms]
term_embeddings = model.encode(term_texts, convert_to_tensor=True)

# Step 3: Encode question
question = "Who is the author of the paper?"
question_embedding = model.encode(question, convert_to_tensor=True)

# Step 4: Compute similarity
cos_scores = util.cos_sim(question_embedding, term_embeddings)[0]
top_results = cos_scores.topk(5)
for score, idx in zip(top_results.values, top_results.indices):
    uri, desc = terms[idx]



In [40]:

for score, idx in zip(top_results.values, top_results.indices):
    uri, desc = terms[idx]
    print(f"URI: {uri}")
    print(f"Description: {desc}")
    print(f"Score: {score:.4f}")
    print("---")

URI: file:///c:/Users/Administrator/kg-qa/schema.rdf#authorOf
Description: Property: authorOf. Comment: The creator is the author of the publication.
Score: 0.5394
---
URI: file:///c:/Users/Administrator/kg-qa/schema.rdf#authoredBy
Description: Property: authoredBy. Comment: The publication is authored by the creator.
Score: 0.5217
---
URI: file:///c:/Users/Administrator/kg-qa/schema.rdf#numberOfCreators
Description: Property: numberOfCreators. Comment: The number of creators who created this publication.
Score: 0.4766
---
URI: file:///c:/Users/Administrator/kg-qa/schema.rdf#creatorOf
Description: Property: creatorOf. Comment: The creator of the publication.
Score: 0.4760
---
URI: file:///c:/Users/Administrator/kg-qa/schema.rdf#AuthorSignature
Description: Class: AuthorSignature. Comment: The information that links a publication to an author.
Score: 0.4635
---


In [65]:
list_1=sorted(list(template_representatives.keys()))

In [66]:
list_2=sorted(list(val_template_representatives.keys()))

In [71]:
list_3=sorted(list(test_template_representatives.keys()))

In [69]:
only_in_list1 = list(set(list_1) - set(list_2))
print("In list1 but not in list2:", only_in_list1)

# Elements in list2 but not in list1
only_in_list2 = list(set(list_2) - set(list_1))
print("In list2 but not in list1:", only_in_list2)

In list1 but not in list2: ['TP71']
In list2 but not in list1: ['TC62', 'TC36', 'TP54', 'TP15', 'TC42', 'TC17', 'TC22', 'TP72', 'TP64', 'TP32', 'TP84', 'TP04', 'TC13', 'TC74', 'TP21', 'TC83', 'TC03', 'TP11', 'TC52']


In [74]:
only_in_list3 = list(set(list_3) - set(list_1))
print("In list3 but not in list1", only_in_list3)


In list3 but not in list21 ['TC62', 'TC36', 'TP54', 'TP15', 'TC42', 'TC17', 'TC22', 'TP72', 'TP64', 'TP32', 'TP84', 'TP04', 'TC13', 'TC74', 'TP21', 'TC83', 'TC03', 'TP11', 'TC52']


In [ ]:
from datasets import load_dataset
from utils import title_to_filename

dataset = load_dataset("allenai/qasper")
df = dataset["train"].to_pandas()
df['qas'].iloc[0]['question'][0]
# for row in df:
#     filename = title_to_filename(row['title'])
#     question = row['question']
#     print(question)

TypeError: string indices must be integers, not 'str'

'What is the seed lexicon?'

In [11]:
import os

for i, row in df.iterrows():
    title = row["title"]
    filename = title_to_filename(title)
    pdf_path = f"pdfs/{filename}.pdf"
        
    if not os.path.exists(pdf_path):
        print(f"PDF not found: {pdf_path}")
        continue
        
        # Get the first question
    try:
        question = row["qas"]["question"][0]
    except (IndexError, KeyError, TypeError):
        print(f" No valid question found for: {title}")
        continue

PDF not found: pdfs/po_emo_conceptualization_annotation_and_modeling_of_aesthetic_emotions_in_german_and_english_poetry.pdf
PDF not found: pdfs/progress_and_tradeoffs_in_neural_language_models.pdf
PDF not found: pdfs/stay_on_topic_generating_context_specific_fake_restaurant_reviews.pdf
PDF not found: pdfs/saliency_maps_generation_for_automatic_text_summarization.pdf
PDF not found: pdfs/probabilistic_bias_mitigation_in_word_embeddings.pdf
PDF not found: pdfs/massive_vs_curated_word_embeddings_for_low_resourced_languages_the_case_of_yor_ub_a_and_twi.pdf
PDF not found: pdfs/is_there_gender_bias_and_stereotype_in_portuguese_word_embeddings.pdf
PDF not found: pdfs/citation_data_of_czech_apex_courts.pdf
PDF not found: pdfs/comprehensive_named_entity_recognition_on_cord_19_with_distant_or_weak_supervision.pdf
PDF not found: pdfs/unisent_universal_adaptable_sentiment_lexica_for_1000_languages.pdf
PDF not found: pdfs/word_sense_disambiguation_for_158_languages_using_word_embeddings_only.pdf
PDF